In [1]:
import anndata as ad
from pycombat import Combat
import numpy as np
import pandas as pd
import os

In [7]:
correspondence = {}
correspondence['149Sm_CREB'] ='149Sm_pCREB'
correspondence['167Er_ERK'] ='167Er_pERK12'
correspondence['164Dy_IkB'] ='164Dy_IkB'
correspondence['159Tb_MAPKAPK2'] ='159Tb_pMAPKAPK2'
correspondence['166Er_NFkB'] ='166Er_pNFkB'
correspondence['151Eu_p38'] ='151Eu_pp38'
correspondence['155Gd_S6'] ='155Gd_pS6'
correspondence['153Eu_STAT1'] ='153Eu_pSTAT1'
correspondence['154Sm_STAT3'] ='154Sm_pSTAT3'
correspondence['150Nd_STAT5'] ='150Nd_pSTAT5'
correspondence['168Er_pSTAT6'] ='168Yb_pSTAT6'
correspondence['174Yb_HLADR'] ='174Yb_HLADR'
correspondence['169Tm_CD25'] ='169Tm_CD25'

In [11]:
def concat_write_data(unstim_corrected,stim_corrected,stim,data_path,directory):
    data_corrected=ad.concat(
        {"Unstim": unstim_corrected, stim: stim_corrected},
        label="stim"
    )

    data_corrected = data_corrected.copy()
    data_corrected.strings_to_categoricals()

    for col in data_corrected.obs.columns:
        if isinstance(data_corrected.obs[col].dtype, pd.CategoricalDtype):
            data_corrected.obs[col] = data_corrected.obs[col].astype(str)
    surge_name= data_path.split('/')[-1]
    data_corrected.write(directory+surge_name)
    return

In [24]:
def batch_correction_combat(correspondence,perio_path,surge_path,stim):
    perio = ad.read_h5ad(perio_path)
    surge = ad.read_h5ad(surge_path)

    if stim=='LPS':
        perio_stim='P. gingivalis'
    else:
        perio_stim=stim
    # Load AnnData
    perio_unstim=perio[perio.obs['drug']=='Unstim']
    surge_unstim=surge[surge.obs['drug']=='Unstim']

    surge_stim=surge[surge.obs['drug']==stim]
    perio_stim=perio[perio.obs['drug']==perio_stim]

    # Choose the features
    features=correspondence.values()
    features = list(features)

    surge_unstim = surge_unstim[:, features]
    surge_stim = surge_stim[:, features]

    # Rename the features in perio_unstim to match those in surge_unstim
    perio_unstim.var_names = perio_unstim.var_names.to_series().replace(correspondence).values
    perio_stim.var_names = perio_stim.var_names.to_series().replace(correspondence).values
    perio_unstim = perio_unstim[:, features]
    perio_stim = perio_stim[:, features]

    assert perio_unstim.var_names.equals(surge_unstim.var_names)
    
    combined_unstim = ad.concat(
        {"perio": perio_unstim, "surge": surge_unstim},
        label="batch"
    )

    combat_unstim = Combat()
    X_corrected_unstim = combat_unstim.fit_transform(combined_unstim.X, combined_unstim.obs["batch"])

    surge_unstim_corrected = surge_unstim.copy()
    surge_unstim_corrected.X = X_corrected_unstim[combined_unstim.obs["batch"] == "surge"]

    perio_unstim_corrected = perio_unstim.copy()
    perio_unstim_corrected.X = X_corrected_unstim[combined_unstim.obs["batch"] == "perio"]

    combined_stim = ad.concat(
        {"perio": perio_stim, "surge": surge_stim},
        label="batch"
    )

    combat_stim = Combat()
    X_corrected_stim = combat_stim.fit_transform(combined_stim.X, combined_stim.obs["batch"])

    surge_stim_corrected = surge_stim.copy()
    surge_stim_corrected.X = X_corrected_stim[combined_stim.obs["batch"] == "surge"]

    perio_stim_corrected = perio_stim.copy()
    perio_stim_corrected.X = X_corrected_stim[combined_stim.obs["batch"] == "perio"]

    directory_perio= 'perio_dbl_corrected2/'
    directory_surge= 'surge_dbl_corrected2/'
    if not os.path.exists(directory_perio):
        os.makedirs(directory_perio)
    if not os.path.exists(directory_surge):
        os.makedirs(directory_surge)
    concat_write_data(perio_unstim_corrected,perio_stim_corrected,stim,perio_path,directory_perio)
    concat_write_data(surge_unstim_corrected,surge_stim_corrected,stim,surge_path,directory_surge)

    return combat_unstim,combat_stim

In [14]:
def apply_combat(to_correct,combat):
    dummy_to_correct = to_correct[:1].copy()  # copy one row from surge_stim
    dummy_to_correct.X = np.zeros_like(dummy_to_correct.X)  # zero values
    dummy_to_correct.obs['batch'] = 'perio'

    combined_data = ad.concat(
        {"surge": to_correct, "perio": dummy_to_correct},
        label="batch"
    )

    # Perform batch correction on the combined data
    X_stim_corrected = combat.transform(combined_data.X,combined_data.obs['batch'])  # Only transform, don't fit
    batch_data_corrected = to_correct.copy()
    batch_data_corrected.X = X_stim_corrected[combined_data.obs["batch"] == "surge"]
    return batch_data_corrected

In [25]:
directory_perio='perio_with_patient'
directory_surge_train='surge_concatenated_train'
directory_surge_test='surge_concatenated_test'

listfile_perio=[f for f in os.listdir(directory_perio) if 'IFNa' not in f]
listfile_surge_train=[f for f in os.listdir(directory_surge_train) if 'IFNa' not in f]
listfile_surge_test=[f for f in os.listdir(directory_surge_test) if 'IFNa' not in f]

listfile_perio.sort()
listfile_surge_train.sort()
listfile_surge_test.sort()

for filename_perio,filename_surge_train,filename_surge_test in zip(listfile_perio,listfile_surge_train,listfile_surge_test):
    path_perio=os.path.join(directory_perio, filename_perio)
    path_surge_train=os.path.join(directory_surge_train, filename_surge_train)
    path_surge_test=os.path.join(directory_surge_test, filename_surge_test)
    if 'LPS' in filename_surge_train:
        stim='LPS'
    else:
        stim='TNFa'
    combat_unstim,combat_stim=batch_correction_combat(correspondence, path_perio, path_surge_train,stim)
    # Apply combat to the surge data
    surge_test = ad.read_h5ad(path_surge_test)
    surge_unstim=surge_test[surge_test.obs['drug']=='Unstim']

    surge_stim=surge_test[surge_test.obs['drug']==stim]

    # Choose the features
    features=correspondence.values()
    features = list(features)

    surge_unstim = surge_unstim[:, features]
    surge_stim = surge_stim[:, features]

    surge_unstim_corrected = apply_combat(surge_unstim, combat_unstim)
    surge_stim_corrected = apply_combat(surge_stim, combat_stim)
    direc_surge_test='surge_dbl_corrected_test/'
    concat_write_data(surge_unstim_corrected,surge_stim_corrected,stim,path_surge_test,direc_surge_test)

    print(f"Corrected {filename_perio} and {filename_surge_train}")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

Corrected perio_data_sherlockP._gingivalis_B-Cells_(CD19+CD3-).h5ad and surge_concatenated_LPS_B-Cells (CD19+CD3-).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

Corrected perio_data_sherlockP._gingivalis_CD4_T-Cells.h5ad and surge_concatenated_LPS_CD4 T-Cells.h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'patient' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation

...

Corrected perio_data_sherlockTNFa_Non-classical_Monocytes_(CD14-CD16+).h5ad and surge_concatenated_TNFa_Non-classical Monocytes (CD14-CD16+).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

Corrected perio_data_sherlockTNFa_Tregs_(CD25+FoxP3+).h5ad and surge_concatenated_TNFa_Tregs (CD25+FoxP3+).h5ad


... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'patient' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To ma

Corrected perio_data_sherlockTNFa_mDCs_(CD11c+HLADR+).h5ad and surge_concatenated_TNFa_mDCs (CD11c+HLADR+).h5ad
Corrected perio_data_sherlockTNFa_pDCs(CD123+HLADR+).h5ad and surge_concatenated_TNFa_pDCs(CD123+HLADR+).h5ad
